In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd

# Load the CSV file containing the wallet addresses
df_wallets = pd.read_csv('/kaggle/input/zeru-ai-engineer-problem2-wallet-id/Wallet id - Sheet1.csv')  # Replace with your actual filename
wallets = df_wallets['wallet_id'].dropna().tolist()

print(f"Loaded {len(wallets)} wallet addresses.")

Loaded 103 wallet addresses.


In [ ]:
import pandas as pd
import random
from tqdm import tqdm

# Step 1: Load Wallets from CSV

wallets_df = pd.read_csv("wallets.csv")
wallets = wallets_df["wallet_id"].dropna().unique().tolist()
print(f"Loaded {len(wallets)} wallets.")

# Step 2: Simulated Transaction Fetcher

def simulate_compound_activity(wallet):
    """
    Mock API: Returns fake transaction metrics per wallet
    In actual use, replace this with Covalent or DefiLlama API calls.
    """
    random.seed(wallet)  # deterministic per wallet
    return {
        "supply_volume": random.uniform(0, 10000),
        "borrow_volume": random.uniform(0, 8000),
        "repay_volume": random.uniform(0, 9000),
        "liquidation_count": random.randint(0, 5),
        "total_txns": random.randint(5, 100),
    }


# Step 3: Feature Extraction Loop

all_data = []
for wallet in tqdm(wallets):
    data = simulate_compound_activity(wallet)
    data["wallet_id"] = wallet
    all_data.append(data)

df = pd.DataFrame(all_data)

# Step 4: Feature Normalization + Scoring

score_features = ['supply_volume', 'borrow_volume', 'repay_volume', 'total_txns', 'liquidation_count']

# Normalize features (manual min-max scaling with fallback)
for col in score_features:
    min_val = df[col].min()
    max_val = df[col].max()
    if max_val > min_val:
        df[col + '_norm'] = (df[col] - min_val) / (max_val - min_val)
    else:
        df[col + '_norm'] = 0.0

# Compute weighted score (higher liquidation = worse)
df['score_raw'] = (
    0.25 * df['supply_volume_norm'] +
    0.25 * df['repay_volume_norm'] +
    0.20 * df['borrow_volume_norm'] +
    0.10 * df['total_txns_norm'] +
    0.20 * (1 - df['liquidation_count_norm'])
)

# Scale to 0–1000
min_score = df['score_raw'].min()
max_score = df['score_raw'].max()
if max_score > min_score:
    df['score'] = ((df['score_raw'] - min_score) / (max_score - min_score) * 1000).astype(int)
else:
    df['score'] = 200  # fallback


# Step 5: Export CSV

df[['wallet_id', 'score']].to_csv("wallet_scores.csv", index=False)
print("✅ Done! Saved to `wallet_scores.csv`")


# Optional Debug Print

print(df[['wallet_id', 'supply_volume', 'borrow_volume', 'repay_volume', 'liquidation_count', 'total_txns', 'score']].head())